# Introduction

This is a housing "Rent vs. Buy" calculator.  Like https://www.nytimes.com/interactive/2014/upshot/buy-rent-calculator.html and https://smartasset.com/mortgage/rent-vs-buy, the goal is to show the financial consequences of a housing decision.  This one adds realism in several ways:

- Directs the renter to buy funds or stocks with the cash flow that renting liberates.  For example, if owning costs \$5,000/mon and renting costs \$3,000/mon, the rent model will invest \$2,000/mon to close the gap.  (That is in addition to investing an amount equal to a down payment.)
- Simulates more tax behavior.  This includes the [net investment income tax](https://www.irs.gov/individuals/net-investment-income-tax) and opportunities lost to the standard deduction, the [$10,000 limit on federally-deductible state taxes](https://www.irs.gov/taxtopics/tc503), the [\$250,000/\$500,000 home sale gain exclusion](https://www.irs.gov/taxtopics/tc701), and the [\$750,000 mortgage indebtedness deduction limit](https://www.irs.gov/publications/p936), and [California's \$1,000,000 mortgage indebtedness deduction limit](https://www.ftb.ca.gov/forms/2024/2024-540-ca-instructions.html).
- Models the possibility that the person will donate assets or die before liquidating them, thereby avoiding capital gains tax.
- Acquires market data history for user-provided ticker symbols.
- Prints enough details to be auditable.

The trade-off is that this requires significantly more user input.  If the New York Times calculator convinces you to rent, you may as well skip this calculator.  If that calculator suggests buying, try this one for a higher-precision estimate.

# Usage

- For the latest code, visit https://colab.research.google.com/github/nmisch/bearable-finance-models/blob/main/Buy_vs_Rent.ipynb.
- (optional) Use File -> Save a copy in Drive.  If you do that once, Colab will save later changes automatically.
- Edit section **Parameters**, below, to fit the rent/buy decision you have in view.  Updating FRED_API_KEY is the minimum; all other parameters have usable defaults.
- Use "Run All" in the "Runtime" menu.
- Visit the **Results** section at the bottom of the page.  After a few minutes, **Statistics** and a graph will appear.  If in doubt, start with the mean and standard deviation in the table of statistics.

# Limitations

- Models mortgages as fixed-rate, but proposes rates typical of adjustable-rate mortgages.
- Models tax constants as fixed across years.  For tax rate tables and other figures to which a government applies inflation adjustments automatically, this model should do the same.
- Does not model refinancing strategies.
- Realizes tax savings monthly.  Ideally, one should realize them on the date when an actual tax payment is lower.
- Invests and settles payables on the first of each month.  Payments like insurance happen once per year in the model, but some insurers bill on different schedules.
- Uses a constant ratio of repairs to capital improvements.  In practice, that ratio increases over time, because your cost basis can include only one roof, one HVAC system, etc. (["basis doesn't include the cost of any improvements that are replaced"](https://www.irs.gov/publications/p530)).
- Treats all dividends and capital gains distributions as [qualified dividends](https://www.irs.gov/publications/p550).  (Does not distinguish dividends on positions held less than 60 days.  Does not predict [payments in lieu of dividends](https://www.fidelity.com/tax-information/tax-topics/annual-credit-for-substitute-payments).  Does not model [tax-exempt securities](https://www.irs.gov/publications/p550).)
- Assumes California.  In particular, residents of US states with low income tax may be able to deduct property taxes federally.  This model has the state tax capital gains like ordinary income.  That holds for CA and NY, but some states, e.g. [MA](https://www.mass.gov/service-details/tax-rates), have distinct capital gains tax rates.  Outside the US, there will be a lot to change.
- Does not attempt to calculate \$X in "If you can rent a dwelling for less than \$X, renting is cheaper."
- Does not give a sense of total cost.  You'll take either the rent scenario or the buy scenario.  What matters is the relative financial positions of those two.

# Development

Note that changes reducing the recurring costs of ownership in this model may leave the ownership liquidation value unchanged and reduce the rent liquidation value.  This is a consequence of the renter investing based on the owner's expenses.

See https://github.com/nmisch/bearable-finance-models for development history.  If you find ways to remove limitations or otherwise improve this model, please send a pull request.

# License

Copyright 2025 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [ ]:
!pip install attrs yfinance fredapi numpy_financial

# Tax Constants
These are the same for everyone, but governments change them.  Last updated 2025-11.

In [ ]:
import enum

class FilingStatus(enum.Enum):
  SINGLE = 1
  MARRIED = 2  # filing jointly
  # TODO: less-common statuses

# Maximum amount of state tax deductible at the federal level.
# As of 2025-11-09; not adjusted annually.  Deduction is higher, up to $40,000,
# for incomes below $600,000.  TODO implement the calculation per
# https://www.irs.gov/pub/irs-dft/f1040sa--dft.pdf
federal_deductions_ceiling_from_state_tax = 10000

# Maximum amount of mortgage indebtedness that can produce deductible interest.
# as of 2025-11-09; not adjusted annually
federal_mortgage_deduction_ceiling = 750000
state_mortgage_deduction_ceiling = 1e6

# for tax year 2026; updated in Q4 of the PRIOR year; from:
# https://www.irs.gov/newsroom/irs-releases-tax-inflation-adjustments-for-tax-year-2026-including-amendments-from-the-one-big-beautiful-bill
# Left-hand value is the last dollar taxed at a lower rate
federal_marginal_rates = {
    FilingStatus.SINGLE: [(0, 0.10),
                          (12400, 0.12),
                          (50400, 0.22),
                          (105700, 0.24),
                          (201775, 0.32),
                          (256225, 0.35),
                          (640600, 0.37)],
    FilingStatus.MARRIED: [(0, 0.10),
                           (24800, 0.12),
                           (100800, 0.22),
                           (211400, 0.24),
                           (403550, 0.32),
                           (512450, 0.35),
                           (768700, 0.37)],
}

# for tax year 2024; updated in Q4 of the TAX year; from:
# https://web.archive.org/web/20241116221018/https://www.ftb.ca.gov/about-ftb/newsroom/tax-news/october-2024/index.html
# 2025 rates not yet available as of 2025-11-09
# Left-hand value is the last dollar taxed at a lower rate
ca_marginal_tax_rates = {
    FilingStatus.SINGLE: [(0, 0.01),
                          (10756, 0.02),
                          (25499, 0.04),
                          (40245, 0.06),
                          (55866, 0.08),
                          (70606, 0.093),
                          (360659, 0.103),
                          (432787, 0.113),
                          (721314, 0.123)],
    FilingStatus.MARRIED: [(0, 0.01),
                           (21512, 0.02),
                           (50998, 0.04),
                           (80490, 0.06),
                           (111732, 0.08),
                           (141212, 0.093),
                           (721318, 0.103),
                           (865574, 0.113),
                           (1442628, 0.123)],
}

# for tax year 2026; updated in Q4 of the PRIOR year; from:
# https://www.irs.gov/irb/2025-45_IRB
# find via web search: bulletin "Maximum Capital Gains Rate" site:irs.gov
federal_lt_capital_gains_rates = {
    FilingStatus.SINGLE: [(0,0),
                          (49450, 0.15),
                          (545500, 0.20)],
    FilingStatus.MARRIED: [(0,0),
                           (98900, 0.15),
                           (613700, 0.20)],
}

net_investment_income_tax_rates = {  # as of 2025-11-09; not adjusted annually
    FilingStatus.SINGLE: [(0,0),
                          (200000, 0.038)],
    FilingStatus.MARRIED: [(0,0),
                           (250000, 0.038)],
}

additional_medicare_tax_rates = {  # as of 2025-11-09; not adjusted annually
    FilingStatus.SINGLE: [(0,0),
                          (200000, 0.009)],
    FilingStatus.MARRIED: [(0,0),
                           (250000, 0.009)],
}

# for tax year 2026; updated in Q4 of the PRIOR year; from:
# https://www.irs.gov/newsroom/irs-releases-tax-inflation-adjustments-for-tax-year-2026-including-amendments-from-the-one-big-beautiful-bill
federal_std_deductions = {
    FilingStatus.SINGLE: 16100,
    FilingStatus.MARRIED: 32200,
}

# for tax year 2024; updated in Q4 of the TAX year; from:
# https://web.archive.org/web/20241116221018/https://www.ftb.ca.gov/about-ftb/newsroom/tax-news/october-2024/index.html
# 2025 rates not yet available as of 2025-11-09
ca_std_deductions = {
    FilingStatus.SINGLE: 5540,
    FilingStatus.MARRIED: 11080,
}

# for tax year 2024; updated in Q1 of the year AFTER the tax year; search for
# "AGI Limitation" in https://www.ftb.ca.gov/forms/2024/2024-540-booklet.html
ca_itemized_thresholds = {
    FilingStatus.SINGLE: 244857,
    FilingStatus.MARRIED: 489719,
}

federal_house_gain_exclusion = {  # as of 2025-11-09; not adjusted annually
    FilingStatus.SINGLE: 250000,
    FilingStatus.MARRIED: 500000,
}

# Parameters

This section customizes the model to the inputs you provide here.  If you plan to share this notebook or its outputs with others, consider rounding sensitive values.

## Housing Facts

In [ ]:
# get one at https://fred.stlouisfed.org/docs/api/api_key.html
FRED_API_KEY = 'insert_your_own_key'
assert (
    FRED_API_KEY != 'insert_your_own_key'
), 'sign up at https://fred.stlouisfed.org/docs/api/api_key.html'
# Years you plan to own and occupy the house.  Longer terms reduce the relative
# loss from closing costs.  Longer terms increase the relative loss from income
# tax on the portion of capital gain exceeding the $250,000/$500,000 home sale
# gain exclusion.
hold_term_years = 10
# Mortgage term is typically 15 or 30, depending on your mortgage product.
# "mortgage_term_years = float('inf')" models a non-amortizing loan.  Such loans
# are not generally available, but you could mimic one by taking a cash-out
# refinance as often as it makes sense.  If the goal were to beat the rent
# scenario on average financial return, a non-amortizing loan would be the best
# kind of loan.
mortgage_term_years = 30
# If your non-housing, non-investment spending will be higher when renting,
# raise the rent by that amount.  For example, some renters spend more on
# vacations or restaurants than they would as owners.
rent = 3660
security_deposit = rent
rent_broker_fee = 800
# https://www.redfin.com/CA/Mountain-View/2005-Showers-Dr-94040/home/12172125
purchase = 1.35e6
# mortgage_rate_locked=None directs the model to predict your rate from
# macroeconomic data; see section "Loan Rates" for details.  You can override
# that by entering a specific rate and deleting mortgage_rate_locked=None.  This
# is useful if you have a locked-in rate or want to examine the past as though
# the low rates of 2020-2021 had been available.
mortgage_rate_locked = 0.026  # 2.6%
mortgage_rate_locked = None
down_payment = 0.20 * purchase
property_tax_rate = 0.0121
buyer_closing_cost_frac = 0.04
seller_closing_cost_frac = 0.06
property_tax_appreciation_rate = 0.02  # proposition 13
rent_insurance_per_year = 200
# https://www.nerdwallet.com/article/insurance/average-homeowners-insurance-cost
own_insurance_per_year = 1150
hoa_per_month = 238  # dues to homeowners association, if any
landlord_paid_utilities_per_month = 50  # treated as raising cost of owning
repair_per_year = 7000  # non-capital repairs
# Capital improvements (roof, siding, etc.).  For condos, include an estimate of
# special assessments.
#
# TODO ratio of repairs to capital improvements increases over time, since only
# one HVAC system, roof, etc. can add to your basis.  This is largely negligible
# at the ten-year scale.
capital_improvement_per_year = 3000

## Investment Facts

The rent scenario applies this investment plan to the cash flow that renting liberates.  The buy scenario doesn't use it.  The default is symbol is VFINX, an S&P500 fund with history back to the 1970s.  (Yahoo Finance has more history for ^GSPC, but it has no dividend data for ^GSPC.)

In [ ]:
invest_symbol = 'VFINX'
invest_leverage = 1.0  # set to 1.0 for no leverage

## Tax Facts

The models use these only to estimate the tax rates applicable to capital gains (home sale, investment sale) and to tax deductions (home mortgage interest, investment interest) caused by the modeled activities.

In [ ]:
filing_status = FilingStatus.SINGLE  # or MARRIED

# If in doubt, consult your most recent tax return.  Enter the sum of the
# "Qualified dividends" line of Form 1040 and the "Net long-term capital gain"
# line of Schedule D.
long_term_income = 11000

# If in doubt, consult your most recent tax return.  Subtract long_term_income
# from the "adjusted gross income" line of Form 1040.  Then, subtract the
# "Qualified business income deduction" line of Form 1040.
short_term_income = 190000

# Federal deductions you take due to state taxes, regardless of housing.  This
# is correct for most software engineers in California.
federal_deductions_from_state_tax = federal_deductions_ceiling_from_state_tax

# Deductions you have regardless of the housing decision and exclusive of state
# taxes.  For example, $10000 of charity deductions.
deductions_other = 10000

## Tax Future

Like tax facts, the models use these only to estimate tax rates.

In [ ]:
# Annual growth of short_term_income above.  Consider referencing
# past tax returns.
short_term_income_appreciation_rate = 0.05

# Annual growth of long_term_income above.  Consider referencing past tax
# returns.  The models already add income from modeled activities, so don't add
# that here.
long_term_income_appreciation_rate = 0.12

# Probability of a taxable liquidation of the financial products acquired in
# the rental strategy.  0 means you will certainly hold them until you die or
# donate them.  1.0 means will you certainly liquidate them immediately at the
# end of the simulation period.  If you expect to liquidate from a lower tax
# bracket, you can lower this to approximate that effect.
p_liquidation_invest = 0.2

# Probability of liquidating the real estate at the end of the simulation.  The
# simulation doesn't fully support anything other than liquidating; setting this
# below 1.0 pretends some of the capital gains taxes don't happen.
p_liquidation_dwelling = 0.9

## Housing Market & Inflation Future

Unlike the parameters above, the model sends these symbols to external APIs (yfinance and fredapi) to retrieve historical data.  Change this if targeting a housing market other than MTV/SVL/SJC.

In [ ]:
# https://fred.stlouisfed.org/series/CUURA422SEHA is "Consumer Price Index for
# All Urban Consumers: Rent of Primary Residence in
# San Francisco-Oakland-Hayward, CA (CBSA)".  I don't know of an index closer to
# MTV/SVL.
rent_growth_symbol = 'CUURA422SEHA'

# MTV/SVL/SJC: https://fred.stlouisfed.org/series/ATNHPIUS41940Q
# SFO: https://fred.stlouisfed.org/series/SFXRSA
house_growth_symbol = 'ATNHPIUS41940Q'

# https://fred.stlouisfed.org/series/CUURA422SA0 is "Consumer Price Index for
# All Urban Consumers: All Items in San Francisco-Oakland-Hayward, CA (CBSA)".
# The models use this for any cost not having its own growth symbol.
inflation_symbol = 'CUURA422SA0'

## Loan Rates

Reading this section is optional.  It constitutes the model's theory about how lenders set rates.  [MORTGAGE30US](https://fred.stlouisfed.org/series/MORTGAGE30US) is an average for 30-year fixed mortgages.  Some buyers get excellent rates, not average rates.  Those savings come from relationship discounts, high credit scores, shopping for matches, etc.  Also, many buyers will get adjustable-rate mortgages, but [ARM data history](https://fred.stlouisfed.org/graph/?g=El56) did not start until 2005.  One should not expect a mortgage rate below a risk-free rate, but top borrowers should expect to improve on the typical 1-2% [spread between a risk-free rate and the average mortgage](https://fred.stlouisfed.org/graph/?g=El6q).  I spot-checked a crowdsourced list for fifteen good loans ranging 2016-2021.  Spreads varied widely, but the second-best I saw was 0.21 of the market-wide average spread.  To err on the side of being generous toward the buy scenario, I'm using that multiplier.

TODO Assess spread across larger data sets, not just a spot check.  Take some percentile thereof.

For investment interest, simulate [Interactive Brokers margin rate](https://www.interactivebrokers.com/en/index.php?f=46376&p=m) for balances below $100k.  Given IBKR [Methodology for Determining Effective Rates](https://ibkr.info/article/2949), use [FEDFUNDS](https://fred.stlouisfed.org/series/FEDFUNDS).


In [ ]:
mortgage_symbol = 'MORTGAGE30US'
risk_free_symbol = 'DGS10'
def mortgage_rate(date):
  if mortgage_rate_locked is not None:
    return mortgage_rate_locked
  avg_mortgage = Quote(mortgage_symbol, date)
  risk_free = Quote(risk_free_symbol, date)
  avg_spread = avg_mortgage - risk_free
  excellent_spread = 0.21 * avg_spread
  return 0.01 * (risk_free + excellent_spread)

invest_leverage_cost_symbol = 'FEDFUNDS'
def leverage_cost_rate(date):
  return 0.015 + 0.01 * Quote(invest_leverage_cost_symbol, date)

# Implementation

## Symbol Quotes

In [ ]:
# Pull quote data from APIs

import pandas
import yfinance
from fredapi import Fred

# auto_adjust=False is of paramount importance.  Yahoo publishes two closing
# prices, labeled "Close" and "Adj Close" on pages like these:
# https://finance.yahoo.com/quote/GOOGL/history/?period1=1656633600&period2=1660262400
# https://finance.yahoo.com/quote/VFINX/history/?period1=1622505600&period2=1625097600.
# "Close" adjusts for splits only, while "Adj Close" adjusts for splits,
# dividends, and capital gains.
#
# To judge pre-tax performance of an investment acquired on date $ACQUIRE and
# sold on date $DISPOSE, expect equivalent results from two methods.  (1)
# Compute ratio "Adj Close"@$DISPOSE/"Adj Close"@$ACQUIRE.  (2) Initialize
# $SHARES=1/"Close"@$ACQUIRE.  At each dividend or capital gain distribution
# with $ACQUIRE < $EX_DIVIDEND_DATE <= $DISPOSE, update $SHARES +=
# $SHARES*$distribution_per_share/"Close"@$EX_DIVIDEND_DATE.  The pre-tax
# performance is $SHARES*"Close"@$DISPOSE.  While (2) is more complex, it
# facilitates modeling post-tax performance.  One could also model post-tax
# performance by including dividends and capital gains in taxable income without
# using them to change the share count.
invest_quotes_with_tz = yfinance.Ticker(invest_symbol).history(
    'max', auto_adjust=False)

fred_quotes = {}
fred_symbols = (rent_growth_symbol,
                house_growth_symbol,
                inflation_symbol,
                invest_leverage_cost_symbol,
                mortgage_symbol,
                risk_free_symbol)
for symbol in fred_symbols:
  fred_quotes[symbol] = Fred(api_key=FRED_API_KEY).get_series(symbol).dropna()

In [ ]:
invest_quotes = invest_quotes_with_tz.tz_localize(None)

# Determine the range of months in which every symbol has data.  Trim partial
# months, so code below can ignore that possibility.
oldest_data_month = max(invest_quotes.iloc[0].name,
                        max(x.index[0] for x in fred_quotes.values()))
# If not first of month, use first of next month.
if oldest_data_month.day != 1:
  oldest_data_month += pandas.to_timedelta((oldest_data_month.days_in_month -
                                            oldest_data_month.day + 1),
                                           unit="D")
newest_data_month = min(invest_quotes.iloc[-1].name,
                        min(x.index[-1] for x in fred_quotes.values()))
# If not last day of month, use last day of previous month.
if newest_data_month.day != newest_data_month.days_in_month:
  newest_data_month -= pandas.to_timedelta(newest_data_month.day, unit="D")
print(oldest_data_month, newest_data_month)

In [ ]:
import numpy

print(repr(invest_quotes))
print(type(invest_quotes))
print(type(invest_quotes.index))
print(type(invest_quotes.loc))
print(invest_quotes.columns)

print(repr(fred_quotes[house_growth_symbol]))
print(fred_quotes[house_growth_symbol]['2020-04-01'])
print(type(fred_quotes[house_growth_symbol]))
print(type(fred_quotes[house_growth_symbol]['2020-04-01']))

# The FRED symbols for interest rates have units of percent, so divide the quote
# by 100 before use.
def Quote(symbol, date):
  if symbol == invest_symbol:
    if date in invest_quotes.index:
      return invest_quotes.loc[date]['Close']
    # If the market is closed, interpolate between two points.  This should be
    # doable with invest_quotes.interpolate(), but I gave up on it.
    iprev = invest_quotes.index.get_indexer([date], method='ffill')
    inext = invest_quotes.index.get_indexer([date], method='bfill')
    prev = invest_quotes.iloc[iprev].iloc[0]
    next = invest_quotes.iloc[inext].iloc[0]
    prev_secs = prev.name.timestamp()
    next_secs = next.name.timestamp()
    rate = (next['Close'] - prev['Close']) / (next_secs - prev_secs)
    q = prev['Close'] + rate * (date.timestamp() - prev_secs)
    return q
  else:
    if date in fred_quotes[symbol]:
      return fred_quotes[symbol][date]
    # Similar interpolation.  It's different code because fred returns
    # pandas.Series, while yfinance returns pandas.DataFrame.
    iprev = fred_quotes[symbol].index.get_indexer([date], method='ffill')
    inext = fred_quotes[symbol].index.get_indexer([date], method='bfill')
    prev = fred_quotes[symbol].iloc[iprev].iloc[0]
    next = fred_quotes[symbol].iloc[inext].iloc[0]
    prev_secs = fred_quotes[symbol].index[iprev].asi8[0] / 1e9
    next_secs = fred_quotes[symbol].index[inext].asi8[0] / 1e9
    rate = (next - prev) / (next_secs - prev_secs)
    q = prev + rate * (date.timestamp() - prev_secs)
    return q

# unit tests
for symbol in fred_symbols:
  print(Quote(symbol, pandas.Timestamp('2015-01-01')),
        Quote(symbol, pandas.Timestamp('2019-12-31')),
        Quote(symbol, pandas.Timestamp('2020-01-01')),
        Quote(symbol, pandas.Timestamp('2020-01-02')))
print(Quote(house_growth_symbol, pandas.Timestamp('2007-10-01')),
      Quote(house_growth_symbol, pandas.Timestamp('2007-11-01')),
      Quote(house_growth_symbol, pandas.Timestamp('2007-12-01')),
      Quote(house_growth_symbol, pandas.Timestamp('2008-01-01')))
# compare to: https://finance.yahoo.com/quote/VFINX/history/?period1=1191110400&period2=1199491200
print(Quote(invest_symbol, pandas.Timestamp('2007-10-01')),
      Quote(invest_symbol, pandas.Timestamp('2007-11-01')),
      Quote(invest_symbol, pandas.Timestamp('2007-12-01')),
      Quote(invest_symbol, pandas.Timestamp('2008-01-01')))

## Tax Return

In [ ]:
class IncomeFlag:
  INVEST = 0x1  # https://www.irs.gov/individuals/net-investment-income-tax
  LONG_TERM = 0x2  # qualified dividend or long-term capital gain

# Which return can use this deduction
class DeductFlag:
  FEDERAL = 0x1
  STATE = 0x2

# Compute tax given a tax table.  This corresponds to the "Tax Computation
# Worksheet" in https://www.irs.gov/pub/irs-pdf/i1040gi.pdf (p.78 for the 2020
# edition).
def OneTax(table, taxable):
  ret = 0
  for threshold, rate in reversed(table):
    if taxable <= threshold:
      continue
    ret += rate * (taxable - threshold)
    taxable = threshold
  return ret

class TaxReturn:
  deduction: dict
  income: dict
  return_filing_status: FilingStatus

  def __init__(self, _filing_status=None):
    if _filing_status is None:
      _filing_status = filing_status
    self.deduction = {}
    self.income = {}
    for flags in (DeductFlag.FEDERAL, DeductFlag.STATE,
                  DeductFlag.FEDERAL | DeductFlag.STATE):
      self.deduction[flags] = 0
    for flag1 in (0, IncomeFlag.INVEST):
      for flag2 in (0, IncomeFlag.LONG_TERM):
        self.income[flag1 | flag2] = 0
    self.return_filing_status = _filing_status

  # used when printing the __dict__of an Asset.
  def __repr__(self):
    return str({k: v for k, v in self.__dict__.items()})

  # Record income (negative for a loss), and return the change in tax liability
  # (almost always positive).
  def Earn(self, addend, flags=0):
    was = self._Liability()
    self.income[flags] += addend
    return self._Liability() - was

  # Record a deduction, and return the change in tax liability (negative).
  def Deduct(self, addend, flags=(DeductFlag.FEDERAL | DeductFlag.STATE)):
    was = self._Liability()
    self.deduction[flags] += addend
    return self._Liability() - was

  def _ShortTerm(self):
    agi_without_lt = sum(v for k,v in self.income.items()
                        if not (k & IncomeFlag.LONG_TERM))
    itemized = (
        sum(v for k,v in self.deduction.items() if k & DeductFlag.FEDERAL))
    deduct = max(itemized, federal_std_deductions[self.return_filing_status])
    taxable = max(0, agi_without_lt - deduct)
    return OneTax(federal_marginal_rates[self.return_filing_status],
                  taxable)

  # Like "Qualified Dividends and Capital Gain Tax Worksheet" of
  # https://www.irs.gov/pub/irs-pdf/i1040gi.pdf
  def _LongTerm(self):
    # LT walks the table using total income, but it taxes only LT.
    ret = 0
    agi = sum(v for k,v in self.income.items())
    lt = sum(v for k,v in self.income.items() if k & IncomeFlag.LONG_TERM)
    table = federal_lt_capital_gains_rates[self.return_filing_status]
    for threshold, rate in reversed(table):
      if agi <= threshold:
        continue
      taxable_this_rate = min(lt, agi - threshold)
      ret += rate * taxable_this_rate
      lt -= taxable_this_rate
      agi = threshold
    return ret

  # https://www.irs.gov/individuals/net-investment-income-tax
  def _NetInvestment(self):
    (threshold, rate) = (
        net_investment_income_tax_rates[self.return_filing_status][1])
    taxable = sum(v for k,v in self.income.items() if k & IncomeFlag.INVEST)
    agi = sum(v for k,v in self.income.items())
    ceiling = max(0, agi - threshold)
    return rate * min(taxable, ceiling)

  # No buy or rent scenario changes wages, so ignoring Additional Medicare Tax
  # would not change results.
  def _AdditionalMedicare(self):
    # Assume short-term, non-investment income is wages.
    wages = sum(v for k,v in self.income.items() if k == 0)
    return OneTax(additional_medicare_tax_rates[self.return_filing_status],
                  wages)

  def _Federal(self):
    return (self._ShortTerm() + self._LongTerm() +
            self._NetInvestment() + self._AdditionalMedicare())

  # See "Itemized Deductions Worksheet" in
  # https://www.ftb.ca.gov/forms/2020/2020-540-ca-instructions.html
  def _State(self):
    agi = sum(v for k,v in self.income.items())
    threshold = ca_itemized_thresholds[self.return_filing_status]
    itemized = (
        sum(v for k,v in self.deduction.items() if k & DeductFlag.STATE))
    itemized_disallowed = 0
    if agi > threshold:
      # TODO California isolates deductions for medical expenses, investment
      # interest, and loss to theft.  They aren't reduced this way.  However,
      # they would need to be huge to change the outcome.
      itemized_disallowed = min(0.80 * itemized,
                                0.06 * (agi - threshold))
    deduct = max(itemized - itemized_disallowed,
                 ca_std_deductions[self.return_filing_status])
    taxable = max(0, agi - deduct)
    return OneTax(ca_marginal_tax_rates[self.return_filing_status],
                  taxable)

  def _Liability(self):
    return self._Federal() + self._State()

  def Liability(self):
    return self._Liability()

d = TaxReturn(FilingStatus.SINGLE)
print(d.Earn(90000), "90k-wage")
d = TaxReturn(FilingStatus.MARRIED)
print(d.Earn(90000), "90k-wage-married")
print(d.Deduct(federal_std_deductions[d.return_filing_status] - 50,
               DeductFlag.FEDERAL),
      d.Deduct(100), d.Deduct(100), "mixed-deduct")
d = TaxReturn(FilingStatus.MARRIED)
d.Earn(90000)
d.Deduct(ca_std_deductions[d.return_filing_status] + 1000)
print(d.Deduct(100), "state-only")
d = TaxReturn(FilingStatus.SINGLE)
print(d.Earn(600000), "600k-wage")
print(d.Earn(100, IncomeFlag.INVEST), "NIIT")
d = TaxReturn(FilingStatus.SINGLE); d.Earn(200000 - 10000);
print(d.Earn(100, IncomeFlag.INVEST), "!NIIT")
d = TaxReturn(FilingStatus.SINGLE); d.Earn(200000 - 50);
print(d.Earn(100, IncomeFlag.INVEST), "mixed-NIIT")
# state disallows 80% so state uses std deduction; 37% federal benefit only
d = TaxReturn(FilingStatus.SINGLE); d.Earn(600000)
d.Deduct(15000, DeductFlag.FEDERAL)
print(d.Deduct(10000), "[37%]")
d = TaxReturn(FilingStatus.SINGLE)
d.Earn(500 / .06 + ca_itemized_thresholds[d.return_filing_status])
d.Deduct(ca_std_deductions[d.return_filing_status])
print(d.Deduct(250), d.Deduct(500), "state-disallowed")
# state disallows 80%, but the remaining 20% is large enough to itemize
d = TaxReturn(FilingStatus.SINGLE); d.Earn(1e7)
print(d.Deduct(1e5), "state-80p-disallowed")
d = TaxReturn(FilingStatus.SINGLE); d.Earn(700000);
print(d.Earn(100, IncomeFlag.LONG_TERM | IncomeFlag.INVEST), "lt-invest")
print(d.Earn(100, IncomeFlag.LONG_TERM), "lt")
d = TaxReturn(FilingStatus.MARRIED); d.Earn(79950);
print(d.Earn(100, IncomeFlag.LONG_TERM | IncomeFlag.INVEST), "lt-low-threshold")
d = TaxReturn(FilingStatus.SINGLE); d.Earn(430000);
print(d.Earn(21000, IncomeFlag.LONG_TERM), "lt-hi-threshold")

## Main Model

In [ ]:
from typing import List
import attr
import numpy_financial
import tqdm
import matplotlib.pyplot as plt

def CashTable(cash_dict, include_total=False):
  cols = ('Line Item', '$')
  ret = pandas.DataFrame(cash_dict.items(), columns=cols)
  if include_total:
    total = sum(cash_dict.values())
    ret = pandas.concat(
        [ret, pandas.DataFrame.from_records(
            [{ cols[0]: 'TOTAL', cols[1]: total }])])
  return ret.round(2)

class Verbosity(enum.Enum):
  MONTH = 1  # print cash figures for each month
  SCENARIO = 2
  QUIET = 3  # print errors only

class Asset:
  symbol: str
  shares: float
  cost_basis: float
  liability: float  # debt to retire when selling this asset
  tax: TaxReturn

  def __init__(self, symbol):
    self.symbol = symbol
    self.shares = 0
    self.cost_basis = 0
    self.liability = 0
    self.tax = None

  def Quote(self, date):
    return Quote(self.symbol, date)

  def Buy(self, date, total_cost, commission, liability):
    self.shares += (total_cost - commission) / self.Quote(date)
    self.cost_basis += total_cost
    self.liability += liability

  def BuyLevered(self, date, unlevered_cost):
    assert unlevered_cost > 0
    total_cost = unlevered_cost * invest_leverage
    commission = 0  # close enough for stocks and funds
    self.Buy(date, total_cost, commission, total_cost - unlevered_cost)

  def Sell(self, date, commission_rate, exclude_from_tax, p_taxable, closing):
    gross_sale = self.Quote(date) * self.shares
    commission = commission_rate * gross_sale
    cash_in_hand = gross_sale - commission - self.liability
    capital_gain_or_loss = gross_sale - commission - self.cost_basis
    taxable_gain_or_loss = capital_gain_or_loss
    invest_flag = IncomeFlag.INVEST
    # Houses have a gain exclusion and aren't subject to NIIT, but you also
    # can't deduct a loss.
    if exclude_from_tax:
      taxable_gain_or_loss = max(0, capital_gain_or_loss - exclude_from_tax)
      invest_flag = 0
    else:
      closing['deposit return'] = security_deposit
    tax = self.tax.Earn(taxable_gain_or_loss * p_taxable,
                        IncomeFlag.LONG_TERM | invest_flag)

    closing['gross sale'] = gross_sale
    closing['commission'] = -commission
    closing['debt retirement'] = -self.liability
    closing['tax'] = -tax


class Model:
  anniversary: pandas.Timestamp
  verbosity: Verbosity
  mortgage_rate: float
  inflation_initial_quote: float
  rent_initial_quote: float
  years_elapsed: int
  payment_ordinal: int  # [1, 12*mortgage_term_years]
  queue_cash: dict
  total_cash: dict
  house: Asset
  invest: Asset

  # start TaxReturn for a new calendar year
  def _TaxReturn(self):
    # TODO inflate tax tables, or stop inflating income.
    st_inflate = (1 + short_term_income_appreciation_rate) ** self.years_elapsed
    lt_inflate = (1 + long_term_income_appreciation_rate) ** self.years_elapsed
    ret = TaxReturn(filing_status)
    ret.Earn(short_term_income * st_inflate, 0)
    ret.Earn(long_term_income * lt_inflate, IncomeFlag.LONG_TERM)
    ret.Deduct(federal_deductions_from_state_tax, DeductFlag.FEDERAL)
    ret.Deduct(deductions_other)
    return ret

  def __init__(self, date, verbosity):
    self.anniversary = (date + pandas.to_timedelta(1, unit="D"))
    self.verbosity = verbosity
    self.mortgage_rate = mortgage_rate(date)
    self.inflation_initial_quote = Quote(inflation_symbol, date)
    self.rent_initial_quote = Quote(rent_growth_symbol, date)
    self.years_elapsed = 0
    self.payment_ordinal = 1
    self.queue_cash = {}
    self.total_cash = {}

    closing_costs = buyer_closing_cost_frac * purchase
    self.house = Asset(house_growth_symbol)
    self.house.tax = self._TaxReturn()
    self.house.Buy(date,
                   purchase + closing_costs,
                   closing_costs,
                   purchase - down_payment)
    self._AddCash('down payment', down_payment)
    self._AddCash('buyer closing costs', closing_costs)

    self.invest = Asset(invest_symbol)
    self.invest.tax = self._TaxReturn()
    self._AddCash('rent broker fee', -rent_broker_fee)
    self._AddCash('security deposit', -security_deposit)

  # Log the fact that the ownership scenario spent some cash or, if negative,
  # received some cash.
  def _AddCash(self, label, addend):
    if label in self.queue_cash:
      # Rarely, a security will declare two dividends in one month.  For
      # example, ITB did that in 2012-12.
      assert 'dividend' in label
      self.queue_cash[label] += addend
    else:
      self.queue_cash[label] = addend

  # total_cash += queue_cash; empty queue_cash
  def _UpdateTotalCash(self):
    for k,v in self.queue_cash.items():
      if k in self.total_cash:
        self.total_cash[k] += v
      else:
        self.total_cash[k] = v
    self.queue_cash = {}

  # Only Month() should call this; otherwise, Month() will overstate margin
  # interest.  (Also, that's roughly how nmisch batched orders.)
  def _SpendCash(self, date):
    if self.verbosity == Verbosity.MONTH:
      print("Investing @", date.date())
      print(CashTable(self.queue_cash, include_total=True))

    total = sum(self.queue_cash.values())
    if total == 0:
      # Renting and owning cost exactly the same this month.
      self._UpdateTotalCash()
    elif total < 0:
      # This month, renting used more cash than buying.  Carry the loss.
      self._AddCash('rent loss deferral', -total)
      self._UpdateTotalCash()
      self._AddCash('rent loss carry', total)
    else:
      self._UpdateTotalCash()
      self.invest.BuyLevered(date, total)

  # Tasks that happen once per year, on the anniversary of purchase.
  def Anniversary(self, date):
    if date != self.anniversary:
      self.years_elapsed += 1
    general_inflation = (Quote(inflation_symbol, date) /
                         self.inflation_initial_quote)
    property_tax_inflation = (
        1 + property_tax_appreciation_rate) ** self.years_elapsed

    capital_improvement = capital_improvement_per_year * general_inflation
    self.house.cost_basis += capital_improvement
    self._AddCash('capital improvements', capital_improvement)

    # TODO deduct property tax if not maxed out (hopeless in CA)
    self._AddCash('property tax',
                  property_tax_rate * purchase * property_tax_inflation)
    self._AddCash('home insurance', own_insurance_per_year * general_inflation)
    self._AddCash('repairs', repair_per_year * general_inflation)
    self._AddCash('renters insurance',
                  -rent_insurance_per_year * general_inflation)

  def Year(self, date):
    self.house.tax = self._TaxReturn()
    self.invest.tax = self._TaxReturn()

  # One pays deductible interest monthly, but accruing deductions monthly is the
  # wrong thing.  Mortgage interest decreases over the year, and margin interest
  # increases.  Similarly, if a deduction changes one's marginal tax bracket,
  # that will look like less tax savings later in the year.  Tax payment timing
  # is independent of those effects.
  def Month(self, date):
    if date.month == 1:
      self.Year(date)
    if date.month == self.anniversary.month:
      self.Anniversary(date)

    if numpy.isinf(mortgage_term_years):  # non-amortizing loan
      mortgage_payment = self.house.liability * self.mortgage_rate / 12
      mortgage_principal = 0
    elif self.payment_ordinal > mortgage_term_years * 12:  # loan paid off
      mortgage_payment = 0
      mortgage_principal = 0
    else:
      mortgage_payment = -numpy_financial.pmt(
          self.mortgage_rate / 12,
          mortgage_term_years * 12, purchase - down_payment)
      mortgage_principal = -numpy_financial.ppmt(
          self.mortgage_rate / 12, self.payment_ordinal,
          mortgage_term_years * 12, purchase - down_payment)
      assert mortgage_principal < mortgage_payment

    self.payment_ordinal += 1
    mortgage_interest = mortgage_payment - mortgage_principal
    federal_mortgage_deduction = mortgage_interest
    if self.house.liability > federal_mortgage_deduction_ceiling:
      federal_mortgage_deduction *= (federal_mortgage_deduction_ceiling /
                                     self.house.liability)
    state_mortgage_deduction = mortgage_interest
    if self.house.liability > state_mortgage_deduction_ceiling:
      state_mortgage_deduction *= (state_mortgage_deduction_ceiling /
                                   self.house.liability)
    self.house.liability -= mortgage_principal
    self._AddCash('mortgage principal', mortgage_principal)
    self._AddCash('mortgage interest', mortgage_interest)
    self._AddCash('mortgage interest deduction, federal',
                  self.house.tax.Deduct(federal_mortgage_deduction,
                                        DeductFlag.FEDERAL))
    self._AddCash('mortgage interest deduction, state',
                  self.house.tax.Deduct(state_mortgage_deduction,
                                        DeductFlag.STATE))
    general_inflation = (Quote(inflation_symbol, date) /
                         self.inflation_initial_quote)
    self._AddCash('homeowners association fees',
                  hoa_per_month * general_inflation)
    rent_inflation = (Quote(rent_growth_symbol, date) /
                      self.rent_initial_quote)
    self._AddCash('rent', -rent * rent_inflation)
    self._AddCash('landlord-paid utilities', landlord_paid_utilities_per_month)
    leverage_payment = self.invest.liability * leverage_cost_rate(date) / 12
    self._AddCash('cost of leverage', -leverage_payment)
    self._AddCash('investment interest deduction',
                  -self.invest.tax.Deduct(leverage_payment))
    self._SpendCash(date)

  # This and its callees follows a principle: determine how much owning costs,
  # then spend the same amount while renting, using investment to close any
  # difference.
  def Day(self, date):
    # Queue dividends for reinvestment
    # TODO handle the fact that some dividends are ordinary (!LONG_TERM)
    # TODO don't include div if first day of model; see comment at history()
    if date in invest_quotes.index:
      income_per_share = (invest_quotes.loc[date]['Dividends'] +
                          invest_quotes.loc[date]['Capital Gains'])
      if income_per_share > 0:
        proceeds = self.invest.shares * income_per_share
        self._AddCash('dividends', proceeds)
        self._AddCash(
            'tax on dividends',
            -self.invest.tax.Earn(proceeds,
                                  IncomeFlag.LONG_TERM | IncomeFlag.INVEST))

    if date.day == 1:
      self.Month(date)

  # Liquidate positions.  Returns a dict like {SYMBOL1: LIQUIDATION_VALUE, ...}.
  def Close(self, date):
    ret = {}

    house_closing_cash = {}
    # If we have a rent loss carry, subtract that from invest scenario proceeds.
    invest_closing_cash = self.queue_cash.copy()

    self._UpdateTotalCash()  # no-op unless we have a carried loss

    self.house.Sell(date,
                    seller_closing_cost_frac,
                    federal_house_gain_exclusion[filing_status],
                    p_liquidation_dwelling,
                    house_closing_cash)
    ret[self.house.symbol] = sum(house_closing_cash.values())
    self.invest.Sell(date,
                     0,  # commission is negligible for stocks/ETFs
                     0,  # no gain exclusion
                     p_liquidation_invest,
                     invest_closing_cash)
    ret[self.invest.symbol] = sum(invest_closing_cash.values())
    if self.verbosity != Verbosity.QUIET:
      print("Mortgage rate:", self.mortgage_rate)
      print("Sum of all investing-time cash effects")
      print(CashTable(self.total_cash))
      print(self.house.symbol, "closing @", date.date())
      print(CashTable(house_closing_cash, include_total=True))
      print(self.invest.symbol, "closing @", date.date())
      print(CashTable(invest_closing_cash, include_total=True))

    return ret

# Results

## One Scenario, Verbose Output

In [ ]:
# Run one scenario with maximum output verbosity.
start_date = pandas.Timestamp(year=(2020 - hold_term_years), month=5, day=31)
model = Model(start_date, Verbosity.MONTH)
for date in pandas.date_range(start_date, freq='D',
                              periods=(hold_term_years * 365)):
    model.Day(date)
model.Close(date)

## Generate Full

In [ ]:
# Run the model on each purchase month.
results = {}
for start_date in pandas.date_range(
    pandas.Timestamp(year=(oldest_data_month.year),
                     month=oldest_data_month.month,
                     day=15),
    pandas.Timestamp(year=(newest_data_month.year - hold_term_years),
                     month=newest_data_month.month,
                     day=15),
    freq='M'):
  # start_date is last day of some month
  model = Model(start_date, Verbosity.QUIET)
  for date in pandas.date_range(start_date, freq='D',
                                periods=(hold_term_years * 365)):
    model.Day(date)
  results[start_date] = model.Close(date)

## Statistics

In [ ]:
# Statistics and graphs across the various purchase months.
df = pandas.DataFrame(results).transpose()
df.plot(title=('Owning for %dy vs. renting & investing' %
               (hold_term_years,)),
        xlabel='Date of Dwelling Purchase & Move-in',
        ylabel='$ at Liquidation',
        ylim=(0,None),
        grid=True)
df.describe(percentiles=(.05, .10, .15, .25, .5, .75)).round(2)